In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls /content/drive/My\ Drive/Colab\ Notebooks/lib

per_patient_roc.py  plotting.py  __pycache__  utils.py


In [0]:
import os
import sys 
sys.path.append(os.getcwd() + "/drive/My Drive/Colab Notebooks/lib")
from utils import connectToDB, closeConnection, read_input_doc2vec, findMostSimilarDocumentsDoc2Vec, count_most_similar_doc2vec
import pandas as pd

In [4]:
import numpy as np
import pandas as pd
#import cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [5]:
!python

Python 3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import nltk
>>> nltk.download("punkt")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
True
>>> exit()


In [0]:
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [0]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string.decode('utf-8'))
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/outputs/notes,subjectid,expireflag,chartdate.csv")

In [9]:
df.head()

,Unnamed: 0,subject_id,text,category,chartdate,expire_flag
0,0,2,Neonatology Attending Triage Note\n\nBaby [**N...,Nursing/other,2138-07-17,0
1,1,3,Sinus rhythm\nInferior/lateral ST-T changes ar...,ECG,2101-10-06,1
2,2,3,[**2101-10-6**] 6:02 PM\n CHEST (PORTABLE AP) ...,Radiology,2101-10-06,1
3,3,3,Sinus rhythm\nA-V delay\nNonspecific inferior ...,ECG,2101-10-11,1
4,4,3,[**2101-10-11**] 9:06 PM\n CHEST (PRE-OP PA & ...,Radiology,2101-10-11,1


In [0]:
df.rename(columns = {'text':'notes'}, inplace = True)

In [0]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [12]:
df.head()

,subject_id,notes,category,chartdate,expire_flag
0,2,Neonatology Attending Triage Note\n\nBaby [**N...,Nursing/other,2138-07-17,0
1,3,Sinus rhythm\nInferior/lateral ST-T changes ar...,ECG,2101-10-06,1
2,3,[**2101-10-6**] 6:02 PM\n CHEST (PORTABLE AP) ...,Radiology,2101-10-06,1
3,3,Sinus rhythm\nA-V delay\nNonspecific inferior ...,ECG,2101-10-11,1
4,3,[**2101-10-11**] 9:06 PM\n CHEST (PRE-OP PA & ...,Radiology,2101-10-11,1


In [0]:
df.drop(df.index[[len(df) - 1,len(df) - 2]], inplace = True)

In [17]:
df.notes.shape[0]

197297

In [0]:
from nltk import tokenize

reviews = []
labels = []
texts = []

for idx in range(df.notes.shape[0]):
    text = BeautifulSoup(df.notes[idx])
    text = clean_str(text.get_text().encode('ascii', 'ignore'))
    #text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)

    labels.append(df.expire_flag[idx])

In [19]:
texts[20]

'patient/test information:\nindication: left ventricular function.\nheight: (in) 70\nweight (lb): 215\nbsa (m2): 2.15 m2\nbp (mm hg): 100/60\nhr (bpm): 90\nstatus: inpatient\ndate/time: [**2101-10-21**] at 11:12\ntest: portable tte(complete)\ndoppler: complete pulse and color flow\ncontrast: none\ntechnical quality: adequate\n\n\ninterpretation:\n\nfindings:\n\nleft atrium: the left atrium is mildly dilated.\n\nright atrium/interatrial septum: the right atrium is normal in size.\n\nleft ventricle: left ventricular wall thicknesses are normal. the left\nventricular cavity size is normal. there is severe regional left ventricular\nsystolic dysfunction. overall left ventricular systolic function is severely\ndepressed.\n\nlv wall motion: the following resting regional left ventricular wall motion\nabnormalities are seen: mid anterior - akinetic; mid anteroseptal - akinetic;\nmid inferoseptal - akinetic; mid inferior - akinetic; mid inferolateral -\nakinetic; mid anterolateral - akinetic; 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

In [0]:
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [0]:
for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

In [40]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 166687 unique tokens.


In [0]:
del word_index['and']
del word_index['to']

In [45]:
print('Total %s unique tokens.' % len(word_index))

Total 166685 unique tokens.


In [24]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (197297, 15, 100)
Shape of label tensor: (197297, 2)


In [25]:
len(labels)

197297

In [0]:
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [0]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [28]:
len(x_train) + len(x_val)

197297

In [29]:
print('Number of positive and negative patients in traing and validation set')
print (y_train.sum(axis=0))
print (y_val.sum(axis=0))

Number of positive and negative patients in traing and validation set
[90538. 67300.]
[21128. 18331.]


In [0]:
GLOVE_DIR = "/content/drive/My Drive/Colab Notebooks/glove/glove"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [31]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [0]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)

In [0]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [0]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

In [0]:
review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttLayer(100)(l_lstm_sent)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [0]:
print("model fitting - Hierachical attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - Hierachical attention network
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 157838 samples, validate on 39459 samples
Epoch 1/10
157838/157838 [==============================] - 1851s 12ms/step - loss: 0.4305 - acc: 0.7784 - val_loss: 0.3839 - val_acc: 0.8076
Epoch 2/10
157838/157838 [==============================] - 1825s 12ms/step - loss: 0.3663 - acc: 0.8205 - val_loss: 0.3725 - val_acc: 0.8126
Epoch 3/10
157838/157838 [==============================] - 1854s 12ms/step - loss: 0.3346 - acc: 0.8383 - val_loss: 0.3673 - val_acc: 0.8168
Epoch 4/10
157838/157838 [==============================] - 1873s 12ms/step - loss: 0.3053 - acc: 0.8542 - val_loss: 0.3480 - val_acc: 0.8292
Epoch 5/10
157838/157838 [==============================] - 1866s 12ms/step - loss: 0.2759 - acc: 0.8710 - val_loss: 0.3567 - val_acc: 0.8238
Epoch 6/10
157838/157838 [==============================] - 1860s 12ms/step - loss: 0.2431 - acc: 0.8881 - val_loss: 0.3726 - val_acc: 0.8277
Epoch 7/10
157838/157838 [==============================] - 1865s 12ms/step - loss: 0.2099 - acc:

In [0]:
model.save("/content/drive/My Drive/Colab Notebooks/models/kerasatt_mimic.h5")

In [0]:
model.load_weights("/content/drive/My Drive/Colab Notebooks/models/kerasatt_mimic.h5")

In [70]:
df.head()

,subject_id,notes,category,chartdate,expire_flag
0,2,Neonatology Attending Triage Note\n\nBaby [**N...,Nursing/other,2138-07-17,0
1,3,Sinus rhythm\nInferior/lateral ST-T changes ar...,ECG,2101-10-06,1
2,3,[**2101-10-6**] 6:02 PM\n CHEST (PORTABLE AP) ...,Radiology,2101-10-06,1
3,3,Sinus rhythm\nA-V delay\nNonspecific inferior ...,ECG,2101-10-11,1
4,3,[**2101-10-11**] 9:06 PM\n CHEST (PRE-OP PA & ...,Radiology,2101-10-11,1


In [0]:
train_predict = model.predict(x_train, batch_size=100)[:,0]

In [0]:
val_predict = model.predict(x_val, batch_size=100)[:,0]

In [0]:
df['scores'] = np.concatenate((train_predict,val_predict),axis = 0)

In [0]:
df['scores'] = df['scores'].map(lambda X: (1-X))

In [76]:
df.head()

,subject_id,notes,category,chartdate,expire_flag,scores
0,2,Neonatology Attending Triage Note\n\nBaby [**N...,Nursing/other,2138-07-17,0,2.384186e-07
1,3,Sinus rhythm\nInferior/lateral ST-T changes ar...,ECG,2101-10-06,1,8.839338e-01
2,3,[**2101-10-6**] 6:02 PM\n CHEST (PORTABLE AP) ...,Radiology,2101-10-06,1,9.969654e-01
3,3,Sinus rhythm\nA-V delay\nNonspecific inferior ...,ECG,2101-10-11,1,9.593783e-01
4,3,[**2101-10-11**] 9:06 PM\n CHEST (PRE-OP PA & ...,Radiology,2101-10-11,1,9.994255e-01


In [0]:
df.to_csv("/content/drive/My Drive/Colab Notebooks/outputs/hatt_scores.csv")